In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [47]:
demographics_event= pd.read_csv('/data/public/MLA/VCHAMPS-Test/demographics_event_test.csv').iloc[:,1:]

In [48]:
demographics_event

,Internalpatientid,Age at update,Event date,Marital status,Ruca category
0,100017,85.721080,2009-05-28 01:18:36.0,Married,Rural
1,100041,97.746361,2023-01-06 07:04:39.0,Divorced,Urban
2,100181,71.684971,2003-01-03 20:34:04.0,Married,Urban
3,100186,69.808196,2023-10-24 21:29:46.0,Married,Urban
4,100242,84.905314,2024-03-29 21:24:39.0,Divorced,Rural
...,...,...,...,...,...
34808,99609,89.901179,2017-12-24 21:08:27.0,Widowed,Rural
34809,99674,62.751955,2022-10-17 20:08:58.0,Divorced,Urban
34810,99713,78.949573,2002-02-08 01:39:29.0,Widowed,Rural
34811,99792,67.020799,2013-06-26 02:27:08.0,Married,Urban


In [49]:
demographics_event['Event date'] = demographics_event['Event date'].apply(preprocess_data.preprocess_time_string)

In [50]:
marital_status = {'Married': 0, 'Divorced': 1, 'Never married': 1, 'Separated': 1, 'Single': 1, 'Unknown': float('nan'), 'Widowed': 1, 'Not specified (no value)': float('nan')}
ruca_category = {'Urban': 0,  'Rural': 1, 'Highly rural': 2, 'Not specified': float('nan')}

In [51]:
demographics_event['Marital status encoded'] = preprocess_data.preprocess_substitute_categories(marital_status, 
                                                                               demographics_event['Marital status'])
demographics_event['Ruca category encoded'] = preprocess_data.preprocess_substitute_categories(ruca_category, 
                                                                               demographics_event['Ruca category'])

In [57]:
outpatients_location = pd.read_csv('/home/vivi/FDA_datasets/outpatient_state_test.csv').iloc[:,1:]
demographics_event_location = demographics_event.merge(outpatients_location, how = 'left', on = 'Internalpatientid')

In [59]:
demographics_event_location.head()

,Internalpatientid,Age at update,Event date,Marital status,Ruca category,Marital status encoded,Ruca category encoded,State
0,100017,85.721080,2009-05-28 01:18:36,Married,Rural,0.0,1.0,Indiana
1,100041,97.746361,2023-01-06 07:04:39,Divorced,Urban,1.0,0.0,Minnesota
2,100181,71.684971,2003-01-03 20:34:04,Married,Urban,0.0,0.0,Wisconsin
3,100186,69.808196,2023-10-24 21:29:46,Married,Urban,0.0,0.0,Indiana
4,100242,84.905314,2024-03-29 21:24:39,Divorced,Rural,1.0,1.0,California


In [61]:
demographics_event_location[demographics_event_location['State'].isna()]

,Internalpatientid,Age at update,Event date,Marital status,Ruca category,Marital status encoded,Ruca category encoded,State
4101,36791,68.307207,1998-04-16 06:49:05,Widowed,Urban,1.0,0.0,NaN
5376,104610,70.601812,1994-10-27 23:05:44,Married,Urban,0.0,0.0,NaN
11398,13686,62.138108,2015-12-23 11:29:11,Married,Rural,0.0,1.0,NaN
19853,148175,84.651280,2009-12-20 01:01:26,Not specified (no value),Not specified,NaN,NaN,NaN


In [66]:
demographics_event_location = demographics_event_location.dropna(subset=['State']) # 

In [67]:
demographics_event_location[demographics_event_location['State'].isna()]

,Internalpatientid,Age at update,Event date,Marital status,Ruca category,Marital status encoded,Ruca category encoded,State


In [68]:
state_ruca = demographics_event_location.groupby('State')['Ruca category encoded'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [70]:
for index, row in demographics_event_location.iterrows():
    if pd.isna(row['Ruca category encoded']):
        demographics_event_location.at[index, 'Ruca category encoded'] = state_ruca[state_ruca['State'] == row['State']]['Ruca category encoded']

In [72]:
#demographics_event_location.to_csv('/home/vivi/FDA_datasets/demographics_event_test.csv')